In [15]:
from dotenv import load_dotenv
import os
load_dotenv()
# from read_files import read_text_file
# from langchainutils import create_simple_chain
# from search import searchChunk, searchDocs
# from utils_from_aless import *
import tiktoken
from typing import AsyncIterable
from typing import List, Dict, Tuple, Optional, TypedDict
import re
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.docstore.document import Document
from langchain.output_parsers import (
    OutputFixingParser,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.output_parsers import PydanticOutputParser
from azure.core import MatchConditions
import uuid
import random
from typing import Tuple, List, Dict
import json
from typing import Type, Optional, Dict
# from langchain_core.tools import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun
# from search import searchChunk, searchDocs
from langchain.tools import BaseTool, StructuredTool, tool
import json
from langchain.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.prompts import format_document
from langchain.memory.entity import BaseEntityStore
from datetime import datetime, UTC
from langchain_core.runnables import RunnablePassthrough, Runnable
from typing import List, Dict, Tuple, Optional, AsyncIterable, Any, Mapping
from langchain_core.runnables.base import RunnableSerializable

from langchain_core.output_parsers.base import BaseOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from typing import List, Type, Dict, TypeVar, Any, Union, Optional, NotRequired
from pathlib import Path
from functools import cache
import re

from enum import StrEnum
from langchain.prompts import PromptTemplate

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings, OpenAI, ChatOpenAI
llm = ChatOpenAI(
    model_name="gpt-4o", openai_api_key=os.environ["OPENAI_API_KEY"], openai_organization=os.environ["OPENAI_ORGANIZATION"],
    temperature=0.3,
    streaming=True,
    request_timeout=60)

In [17]:
import os
import ast

def list_python_files(path):
    """
    List all Python files in the given directory path.
    """
    python_files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".py"):
                python_files.append(os.path.join(root, file))
    return python_files


def get_functions_definitions(file_path):
    with open(file_path, 'r', encoding = 'utf-8') as file:
        source_code = file.read()
        tree = ast.parse(source_code)
        function_definitions = []

        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                function_code = ast.get_source_segment(source_code, node)
                function_definitions.append(function_code)

        return source_code

In [18]:
list_python_files("resources")

['resources\\IIRA\\app.py',
 'resources\\IIRA\\__init__.py',
 'resources\\IIRA\\core\\create_analyses.py',
 'resources\\IIRA\\core\\fileinteraction.py',
 'resources\\IIRA\\core\\metrics.py',
 'resources\\IIRA\\core\\__init__.py',
 'resources\\IIRA\\data\\__init__.py',
 'resources\\IIRA\\gui\\analyseframe.py',
 'resources\\IIRA\\gui\\containerframe.py',
 'resources\\IIRA\\gui\\fileframes.py',
 'resources\\IIRA\\gui\\helperframes.py',
 'resources\\IIRA\\gui\\mainframe.py',
 'resources\\IIRA\\gui\\rateframe.py',
 'resources\\IIRA\\gui\\__init__.py']

In [4]:
mainframe = get_functions_definitions("resources/IIRA/gui/mainframe.py")

In [25]:
function_prompt = """Can you write me a detailed docstrings for each function and classes you find in the python file in this path: {name}.
Here is the code: {contenu}.
Please just provide the function name or class name and the docstrings as such: {{'function_name': 'function.py', 'docstring': '..Initializes the main application window and its components..'}}, nothing else. 
Please use google docstring template. I put emphasis on the details of the docstring, please be as detailed and descriptive as possible in the docstrings.
The format to follow: {format_instructions}"""
# class_prompt = """Can you write me a detailed docstring for each classes you find here: {}. 
# Each of the functions of these classes are described by the docstrings: {}"""

def generate_function_doc(names: list[str], prompt:str) -> Dict:
    if not names:
        raise("You forgor bro")
    fonc_defs = []
    
    prompt1 = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert in python coding."),
            ("human", prompt)
        ]
    )
    # Simple chain
    class docy(BaseModel):
        class_or_function: str = Field(description="This field can only be 'class' if it is a class or 'function' if it is a function.")
        class_of_the_function: str = Field(description="When it is a function, we need to know from which class it belongs to. Use the class name of the function.")
        function_name: str = Field(description="Name of the function or class found in the file.")
        docstring: str = Field(description="The very detailed docstring of the function or the class.")
    parser = JsonOutputParser(pydantic_object=docy)

    for name in names:
        prout = get_functions_definitions(name)
        if prout:
            fonc_defs.append({'name': name, 'contenu': prout, 'ai_generated': '', 'format_instructions': parser.get_format_instructions()})

    chain1 = prompt1 | llm | parser

    explains = []
    for dico in fonc_defs:
        explains.append(chain1.invoke(dico))  
    i = 0
    for elem in fonc_defs:
        elem['docstring'] = explains[i]
        i+=1
    return fonc_defs

In [26]:
coucou = generate_function_doc(list_python_files("resources"), function_prompt)

In [27]:
coucou[7]['docstring']

[{'class_or_function': 'class',
  'class_of_the_function': '',
  'function_name': 'MainFrame',
  'docstring': '"""\nMainFrame is the main container frame for the application. It initializes the main application window and its components, including buttons, labels, and frames for different functionalities.\n\nAttributes:\n    container (tk.Tk or tk.Frame): The parent container for this frame.\n"""'},
 {'class_or_function': 'function',
  'class_of_the_function': 'MainFrame',
  'function_name': '__init__',
  'docstring': '"""\nInitializes the MainFrame class. Sets up the main layout, styles, and widgets for the application. If no active profile is found, it prompts the user to create one.\n\nArgs:\n    container (tk.Tk or tk.Frame): The parent container for this frame.\n\nAttributes:\n    menu_bar (ttk.Frame): The menu bar frame.\n    container (ttk.Frame): The main container frame.\n    left_frame (ttk.Frame): The left frame for analysis options.\n    right_frame (ttk.Frame): The right f

In [29]:
example = """ 
<article class="markdown-body entry-content container-lg" itemprop="text"><div class="markdown-heading" dir="auto"><h1 tabindex="-1" class="heading-element" dir="auto">Zircon - A user-friendly Tile Engine &amp; Text GUI <a href="https://twitter.com/intent/tweet?text=Do%20you%20plan%20to%20make%20a%20roguelike%3F%20Look%20no%20further.%20Zircon%20is%20the%20right%20tool%20for%20the%20job.&amp;url=https://github.com/Hexworks/zircon&amp;hashtags=games,roguelikes" rel="nofollow"><img src="https://camo.githubusercontent.com/a00d957c85627f2862ae61b59289e25f875e9ae40a716efdf5c3e032d6b8f863/68747470733a2f2f696d672e736869656c64732e696f2f747769747465722f75726c2f687474702f736869656c64732e696f2e7376673f7374796c653d736f6369616c" alt="Tweet" data-canonical-src="https://img.shields.io/twitter/url/http/shields.io.svg?style=social" style="max-width: 100%;"></a></h1><a id="user-content-zircon---a-user-friendly-tile-engine--text-gui-" class="anchor" aria-label="Permalink: Zircon - A user-friendly Tile Engine &amp; Text GUI " href="#zircon---a-user-friendly-tile-engine--text-gui-"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<p dir="auto"><animated-image data-catalyst=""><a target="_blank" rel="noopener noreferrer nofollow" href="https://camo.githubusercontent.com/e5aa2d5b2fd7abc92d60e14587edcfc7f60f98c7f3e8d3f0b220c7ecf2b8049a/68747470733a2f2f63646e2e646973636f72646170702e636f6d2f6174746163686d656e74732f3230353234353033363038343938353835372f3438313231333030303534303232353535302f66756c6c5f6578616d706c652e676966" data-target="animated-image.originalLink" hidden=""><img src="https://camo.githubusercontent.com/e5aa2d5b2fd7abc92d60e14587edcfc7f60f98c7f3e8d3f0b220c7ecf2b8049a/68747470733a2f2f63646e2e646973636f72646170702e636f6d2f6174746163686d656e74732f3230353234353033363038343938353835372f3438313231333030303534303232353535302f66756c6c5f6578616d706c652e676966" alt="Full Example" data-canonical-src="https://cdn.discordapp.com/attachments/205245036084985857/481213000540225550/full_example.gif" style="max-width: 100%;" data-target="animated-image.originalImage" hidden=""></a>
      <span class="AnimatedImagePlayer" data-target="animated-image.player">
        <a data-target="animated-image.replacedLink" class="AnimatedImagePlayer-images" href="https://camo.githubusercontent.com/e5aa2d5b2fd7abc92d60e14587edcfc7f60f98c7f3e8d3f0b220c7ecf2b8049a/68747470733a2f2f63646e2e646973636f72646170702e636f6d2f6174746163686d656e74732f3230353234353033363038343938353835372f3438313231333030303534303232353535302f66756c6c5f6578616d706c652e676966" target="_blank">
          <span data-target="animated-image.imageContainer">
            <img data-target="animated-image.replacedImage" alt="Full Example" class="AnimatedImagePlayer-animatedImage" src="https://camo.githubusercontent.com/e5aa2d5b2fd7abc92d60e14587edcfc7f60f98c7f3e8d3f0b220c7ecf2b8049a/68747470733a2f2f63646e2e646973636f72646170702e636f6d2f6174746163686d656e74732f3230353234353033363038343938353835372f3438313231333030303534303232353535302f66756c6c5f6578616d706c652e676966">
          </span>
        </a>
        <button data-target="animated-image.imageButton" class="AnimatedImagePlayer-images" tabindex="-1"></button>
        <span class="AnimatedImagePlayer-controls" data-target="animated-image.controls">
          <button data-target="animated-image.playButton" class="AnimatedImagePlayer-button">
            <svg aria-hidden="true" focusable="false" class="octicon icon-play" width="16" height="16" viewBox="0 0 16 16" fill="none" xmlns="http://www.w3.org/2000/svg">
              <path d="M4 13.5427V2.45734C4 1.82607 4.69692 1.4435 5.2295 1.78241L13.9394 7.32507C14.4334 7.63943 14.4334 8.36057 13.9394 8.67493L5.2295 14.2176C4.69692 14.5565 4 14.1739 4 13.5427Z">
            </path></svg>
            <svg aria-hidden="true" focusable="false" class="octicon icon-pause" width="16" height="16" viewBox="0 0 16 16" xmlns="http://www.w3.org/2000/svg">
              <rect x="4" y="2" width="3" height="12" rx="1"></rect>
              <rect x="9" y="2" width="3" height="12" rx="1"></rect>
            </svg>
          </button>
          <a data-target="animated-image.openButton" aria-label="Open in new window" class="AnimatedImagePlayer-button" href="https://camo.githubusercontent.com/e5aa2d5b2fd7abc92d60e14587edcfc7f60f98c7f3e8d3f0b220c7ecf2b8049a/68747470733a2f2f63646e2e646973636f72646170702e636f6d2f6174746163686d656e74732f3230353234353033363038343938353835372f3438313231333030303534303232353535302f66756c6c5f6578616d706c652e676966" target="_blank">
            <svg aria-hidden="true" class="octicon" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" width="16" height="16">
              <path fill-rule="evenodd" d="M10.604 1h4.146a.25.25 0 01.25.25v4.146a.25.25 0 01-.427.177L13.03 4.03 9.28 7.78a.75.75 0 01-1.06-1.06l3.75-3.75-1.543-1.543A.25.25 0 0110.604 1zM3.75 2A1.75 1.75 0 002 3.75v8.5c0 .966.784 1.75 1.75 1.75h8.5A1.75 1.75 0 0014 12.25v-3.5a.75.75 0 00-1.5 0v3.5a.25.25 0 01-.25.25h-8.5a.25.25 0 01-.25-.25v-8.5a.25.25 0 01.25-.25h3.5a.75.75 0 000-1.5h-3.5z"></path>
            </svg>
          </a>
        </span>
      </span></animated-image></p>
<p dir="auto">Need info? Check the <a href="https://hexworks.org/zircon/docs/" rel="nofollow">Docs</a>
| or <a href="https://github.com/Hexworks/zircon/issues/new">Create an issue</a>
| Check <a href="https://github.com/Hexworks/zircon/projects/2">our project Board</a>
| <a href="https://discord.gg/vSNgvBh" rel="nofollow">Ask us on Discord</a>
| Support us on <a href="https://www.patreon.com/hexworks" rel="nofollow">Patreon</a>
| <a href="https://hexworks.github.io/zircon/" rel="nofollow">Javadoc / Kdoc</a></p>
<p dir="auto"><a href="https://circleci.com/gh/Hexworks/zircon/" rel="nofollow"><img src="https://camo.githubusercontent.com/11011b2e148b6590514c6f1ab75418d9f50bd7a9aef23ea33c61b1da008c4f17/68747470733a2f2f636972636c6563692e636f6d2f67682f486578776f726b732f7a6972636f6e2f747265652f6d61737465722e7376673f7374796c653d736869656c64" alt="Circle CI" data-canonical-src="https://circleci.com/gh/Hexworks/zircon/tree/master.svg?style=shield" style="max-width: 100%;"></a>
<a href="https://maven-badges.herokuapp.com/maven-central/org.hexworks.zircon/zircon.core" rel="nofollow"><img src="https://camo.githubusercontent.com/1e7502be51b1148789dc250cb9688fe109231dbea66e5976d9a3d73ea2df29ae/68747470733a2f2f6d6176656e2d6261646765732e6865726f6b756170702e636f6d2f6d6176656e2d63656e7472616c2f6f72672e686578776f726b732e7a6972636f6e2f7a6972636f6e2e636f72652f62616467652e737667" alt="Maven Central" data-canonical-src="https://maven-badges.herokuapp.com/maven-central/org.hexworks.zircon/zircon.core/badge.svg" style="max-width: 100%;"></a>
<a href="https://opensource.org/licenses/Apache-2.0" rel="nofollow"><img src="https://camo.githubusercontent.com/db9dfde8049c5d66ba62fde707d2cfb30e26f9f26ff274c3442c0aec1ec410a4/68747470733a2f2f696d672e736869656c64732e696f2f62616467652f4c6963656e73652d417061636865253230322e302d626c75652e737667" alt="License" data-canonical-src="https://img.shields.io/badge/License-Apache%202.0-blue.svg" style="max-width: 100%;"></a>
<a href="https://github.com/sindresorhus/awesome"><img src="https://camo.githubusercontent.com/50cf39121274b3db22bf1bd72cbe25af9078e037441cb5b5bdef1cc9dc5eb2f7/68747470733a2f2f63646e2e7261776769742e636f6d2f73696e647265736f726875732f617765736f6d652f643733303566333864323966656437386661383536353265336136336531353464643865383832392f6d656469612f62616467652e737667" alt="Awesome" data-canonical-src="https://cdn.rawgit.com/sindresorhus/awesome/d7305f38d29fed78fa85652e3a63e154dd8e8829/media/badge.svg" style="max-width: 100%;"></a></p>
<hr>
<div class="markdown-heading" dir="auto"><h2 tabindex="-1" class="heading-element" dir="auto">Table of Contents</h2><a id="user-content-table-of-contents" class="anchor" aria-label="Permalink: Table of Contents" href="#table-of-contents"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<ul dir="auto">
<li><a href="#getting-started">Getting Started</a>
<ul dir="auto">
<li><a href="#adding-zircon-as-a-maven-dependency">Adding Zircon as a Maven Dependency</a></li>
<li><a href="#basic-usage">Basic Usage</a></li>
</ul>
</li>
<li><a href="#best-practices">Best Practices</a></li>
<li><a href="#features-at-a-glance">Features at a Glance</a>
<ul dir="auto">
<li><a href="#drawing">Drawing</a></li>
<li><a href="#input-handling">Input Handling</a></li>
<li><a href="#layering">Layering</a></li>
<li><a href="#text-gui-components">Text GUI Components</a></li>
<li><a href="#animations">Animations</a></li>
<li><a href="#shape-and-box-drawing">Shape and Box Drawing</a></li>
<li><a href="#fonts-and-tilesets">Fonts and Tilesets</a></li>
<li><a href="#road-map">Road Map</a></li>
<li><a href="#license">License</a></li>
<li><a href="#credits">Credits</a></li>
<li><a href="#thanks">Thanks</a></li>
</ul>
</li>
</ul>
<div class="markdown-heading" dir="auto"><h2 tabindex="-1" class="heading-element" dir="auto">Getting Started</h2><a id="user-content-getting-started" class="anchor" aria-label="Permalink: Getting Started" href="#getting-started"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<p dir="auto">If you want to start working with Zircon you can either add it to your project as a Maven dependency or you can try out
the skeleton projects
(<a href="https://github.com/Hexworks/zircon.skeleton.java">Java</a>, <a href="https://github.com/Hexworks/zircon.skeleton.kotlin">Kotlin</a>)
which come with batteries included.</p>
<p dir="auto">The official <a href="https://hexworks.org/zircon/docs/" rel="nofollow">documentation site</a> contains a lot of information. The examples are also documented on the
<a href="https://hexworks.org/zircon/examples/" rel="nofollow">Zircon Examples</a> page <em>(under construction)</em>, and the best place to start is the
<a href="https://hexworks.org/zircon/docs/2018-07-18-a-zircon-crash-course" rel="nofollow">Zircon Crash Course</a>.</p>
<p dir="auto">If you like learning by doing check out the source of <em>Zircon</em> from <a href="https://github.com/Hexworks/zircon">here</a> and you
can run the examples for yourself. If you are using <em>Java</em>
start <a href="https://github.com/Hexworks/zircon/tree/master/zircon.jvm.examples/src/main/java/org/hexworks/zircon/examples">here</a>
. Alternatively if you use <em>Kotlin</em> the code can be
found <a href="https://github.com/Hexworks/zircon/tree/master/zircon.jvm.examples/src/main/kotlin/org/hexworks/zircon">here</a>.</p>
<p dir="auto">If you just want to peruse the <em>Zircon</em> API navigate <a href="https://github.com/Hexworks/zircon/tree/master/zircon.core/src/commonMain/kotlin/org/hexworks/zircon/api">here</a>. Everything which is intended to be part of the public
API is there.</p>
<p dir="auto">You can find the <em>Javadoc</em> / <em>Kdoc</em> <a href="https://hexworks.github.io/zircon/" rel="nofollow">here</a></p>
<p dir="auto">If you'd like to talk to us, join us on our <a href="https://discord.gg/vSNgvBh" rel="nofollow">Discord Server</a>.</p>
<div class="markdown-heading" dir="auto"><h3 tabindex="-1" class="heading-element" dir="auto">Adding Zircon as a Maven Dependency</h3><a id="user-content-adding-zircon-as-a-maven-dependency" class="anchor" aria-label="Permalink: Adding Zircon as a Maven Dependency" href="#adding-zircon-as-a-maven-dependency"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<p dir="auto">Maven:</p>
<div class="highlight highlight-text-xml notranslate position-relative overflow-auto" dir="auto"><pre>&lt;<span class="pl-ent">dependencies</span>&gt;
    &lt;<span class="pl-ent">dependency</span>&gt;
        &lt;<span class="pl-ent">groupId</span>&gt;org.hexworks.zircon&lt;/<span class="pl-ent">groupId</span>&gt;
        &lt;<span class="pl-ent">artifactId</span>&gt;zircon.core-jvm&lt;/<span class="pl-ent">artifactId</span>&gt;
        &lt;<span class="pl-ent">version</span>&gt;2021.1.0-RELEASE&lt;/<span class="pl-ent">version</span>&gt;
    &lt;/<span class="pl-ent">dependency</span>&gt;
    <span class="pl-c"><span class="pl-c">&lt;!--</span> use zircon.jvm.libgdx if you want to use LibGDX instead of Swing <span class="pl-c">--&gt;</span></span>
    &lt;<span class="pl-ent">dependency</span>&gt;
        &lt;<span class="pl-ent">groupId</span>&gt;org.hexworks.zircon&lt;/<span class="pl-ent">groupId</span>&gt;
        &lt;<span class="pl-ent">artifactId</span>&gt;zircon.jvm.swing&lt;/<span class="pl-ent">artifactId</span>&gt;
        &lt;<span class="pl-ent">version</span>&gt;2021.1.0-RELEASE&lt;/<span class="pl-ent">version</span>&gt;
    &lt;/<span class="pl-ent">dependency</span>&gt;
&lt;/<span class="pl-ent">dependencies</span>&gt;</pre><div class="zeroclipboard-container">
    <clipboard-copy aria-label="Copy" class="ClipboardButton btn btn-invisible js-clipboard-copy m-2 p-0 tooltipped-no-delay d-flex flex-justify-center flex-items-center" data-copy-feedback="Copied!" data-tooltip-direction="w" value="
<dependencies>
    <dependency>
        <groupId>org.hexworks.zircon</groupId>
        <artifactId>zircon.core-jvm</artifactId>
        <version>2021.1.0-RELEASE</version>
    </dependency>
    <!-- use zircon.jvm.libgdx if you want to use LibGDX instead of Swing -->
    <dependency>
        <groupId>org.hexworks.zircon</groupId>
        <artifactId>zircon.jvm.swing</artifactId>
        <version>2021.1.0-RELEASE</version>
    </dependency>
</dependencies>" tabindex="0" role="button">
      <svg aria-hidden="true" height="16" viewBox="0 0 16 16" version="1.1" width="16" data-view-component="true" class="octicon octicon-copy js-clipboard-copy-icon">
    <path d="M0 6.75C0 5.784.784 5 1.75 5h1.5a.75.75 0 0 1 0 1.5h-1.5a.25.25 0 0 0-.25.25v7.5c0 .138.112.25.25.25h7.5a.25.25 0 0 0 .25-.25v-1.5a.75.75 0 0 1 1.5 0v1.5A1.75 1.75 0 0 1 9.25 16h-7.5A1.75 1.75 0 0 1 0 14.25Z"></path><path d="M5 1.75C5 .784 5.784 0 6.75 0h7.5C15.216 0 16 .784 16 1.75v7.5A1.75 1.75 0 0 1 14.25 11h-7.5A1.75 1.75 0 0 1 5 9.25Zm1.75-.25a.25.25 0 0 0-.25.25v7.5c0 .138.112.25.25.25h7.5a.25.25 0 0 0 .25-.25v-7.5a.25.25 0 0 0-.25-.25Z"></path>
</svg>
      <svg aria-hidden="true" height="16" viewBox="0 0 16 16" version="1.1" width="16" data-view-component="true" class="octicon octicon-check js-clipboard-check-icon color-fg-success d-none">
    <path d="M13.78 4.22a.75.75 0 0 1 0 1.06l-7.25 7.25a.75.75 0 0 1-1.06 0L2.22 9.28a.751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018L6 10.94l6.72-6.72a.75.75 0 0 1 1.06 0Z"></path>
</svg>
    </clipboard-copy>
  </div></div>
<p dir="auto">Gradle:</p>
<div class="highlight highlight-source-groovy notranslate position-relative overflow-auto" dir="auto"><pre>dependencies {
    implementation <span class="pl-s"><span class="pl-pds">"</span>org.hexworks.zircon:zircon.core-jvm:2021.1.0-RELEASE<span class="pl-pds">"</span></span>
    implementation <span class="pl-s"><span class="pl-pds">"</span>org.hexworks.zircon:zircon.jvm.swing:2021.1.0-RELEASE<span class="pl-pds">"</span></span>
}</pre><div class="zeroclipboard-container">
    <clipboard-copy aria-label="Copy" class="ClipboardButton btn btn-invisible js-clipboard-copy m-2 p-0 tooltipped-no-delay d-flex flex-justify-center flex-items-center" data-copy-feedback="Copied!" data-tooltip-direction="w" value="dependencies {
    implementation &quot;org.hexworks.zircon:zircon.core-jvm:2021.1.0-RELEASE&quot;
    implementation &quot;org.hexworks.zircon:zircon.jvm.swing:2021.1.0-RELEASE&quot;
}" tabindex="0" role="button">
      <svg aria-hidden="true" height="16" viewBox="0 0 16 16" version="1.1" width="16" data-view-component="true" class="octicon octicon-copy js-clipboard-copy-icon">
    <path d="M0 6.75C0 5.784.784 5 1.75 5h1.5a.75.75 0 0 1 0 1.5h-1.5a.25.25 0 0 0-.25.25v7.5c0 .138.112.25.25.25h7.5a.25.25 0 0 0 .25-.25v-1.5a.75.75 0 0 1 1.5 0v1.5A1.75 1.75 0 0 1 9.25 16h-7.5A1.75 1.75 0 0 1 0 14.25Z"></path><path d="M5 1.75C5 .784 5.784 0 6.75 0h7.5C15.216 0 16 .784 16 1.75v7.5A1.75 1.75 0 0 1 14.25 11h-7.5A1.75 1.75 0 0 1 5 9.25Zm1.75-.25a.25.25 0 0 0-.25.25v7.5c0 .138.112.25.25.25h7.5a.25.25 0 0 0 .25-.25v-7.5a.25.25 0 0 0-.25-.25Z"></path>
</svg>
      <svg aria-hidden="true" height="16" viewBox="0 0 16 16" version="1.1" width="16" data-view-component="true" class="octicon octicon-check js-clipboard-check-icon color-fg-success d-none">
    <path d="M13.78 4.22a.75.75 0 0 1 0 1.06l-7.25 7.25a.75.75 0 0 1-1.06 0L2.22 9.28a.751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018L6 10.94l6.72-6.72a.75.75 0 0 1 1.06 0Z"></path>
</svg>
    </clipboard-copy>
  </div></div>
<div class="markdown-heading" dir="auto"><h3 tabindex="-1" class="heading-element" dir="auto">Basic Usage</h3><a id="user-content-basic-usage" class="anchor" aria-label="Permalink: Basic Usage" href="#basic-usage"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<p dir="auto">Once you have the dependencies set up you can start using <em>Zircon</em> by creating a <code>TileGrid</code>:</p>
<div class="highlight highlight-source-java notranslate position-relative overflow-auto" dir="auto"><pre><span class="pl-k">public</span> <span class="pl-k">class</span> <span class="pl-smi">Main</span> {

    <span class="pl-k">public</span> <span class="pl-k">static</span> <span class="pl-smi">void</span> <span class="pl-en">main</span>(<span class="pl-smi">String</span>[] <span class="pl-s1">args</span>) {

        <span class="pl-c">// a TileGrid represents a 2D grid composed of Tiles</span>
        <span class="pl-smi">TileGrid</span> <span class="pl-s1">tileGrid</span> = <span class="pl-smi">SwingApplications</span>.<span class="pl-en">startTileGrid</span>(
                <span class="pl-smi">AppConfig</span>.<span class="pl-en">newBuilder</span>()
                        <span class="pl-c">// The number of tiles horizontally, and vertically</span>
                        .<span class="pl-en">withSize</span>(<span class="pl-c1">60</span>, <span class="pl-c1">30</span>)
                        <span class="pl-c">// You can choose from a wide array of CP437, True Type or Graphical tilesets</span>
                        <span class="pl-c">// that are built into Zircon</span>
                        .<span class="pl-en">withDefaultTileset</span>(<span class="pl-smi">CP437TilesetResources</span>.<span class="pl-en">rexPaint16x16</span>())
                        .<span class="pl-en">build</span>());

        <span class="pl-c">// A Screen is an abstraction that lets you use text GUI Components</span>
        <span class="pl-c">// You can have multiple Screens attached to the same TileGrid to be able to create multiple</span>
        <span class="pl-c">// screens for your app.</span>
        <span class="pl-smi">Screen</span> <span class="pl-s1">screen</span> = <span class="pl-smi">Screen</span>.<span class="pl-en">create</span>(<span class="pl-s1">tileGrid</span>);

        <span class="pl-c">// Creating text GUI Components is super simple</span>
        <span class="pl-smi">Label</span> <span class="pl-s1">label</span> = <span class="pl-smi">Components</span>.<span class="pl-en">label</span>()
                .<span class="pl-en">withText</span>(<span class="pl-s">"Hello, Zircon!"</span>)
                .<span class="pl-en">withAlignment</span>(<span class="pl-smi">ComponentAlignments</span>.<span class="pl-en">alignmentWithin</span>(<span class="pl-s1">tileGrid</span>, <span class="pl-smi">ComponentAlignment</span>.<span class="pl-c1">CENTER</span>))
                .<span class="pl-en">build</span>();

        <span class="pl-c">// Screens can hold GUI components</span>
        <span class="pl-s1">screen</span>.<span class="pl-en">addComponent</span>(<span class="pl-s1">label</span>);

        <span class="pl-c">// Displaying a screen will make it visible. It will also hide a previously shown Screen.</span>
        <span class="pl-s1">screen</span>.<span class="pl-en">display</span>();

        <span class="pl-c">// Zircon comes with a plethora of built-in color themes</span>
        <span class="pl-s1">screen</span>.<span class="pl-en">setTheme</span>(<span class="pl-smi">ColorThemes</span>.<span class="pl-en">arc</span>());
    }
}</pre><div class="zeroclipboard-container">
    <clipboard-copy aria-label="Copy" class="ClipboardButton btn btn-invisible js-clipboard-copy m-2 p-0 tooltipped-no-delay d-flex flex-justify-center flex-items-center" data-copy-feedback="Copied!" data-tooltip-direction="w" value="public class Main {

    public static void main(String[] args) {

        // a TileGrid represents a 2D grid composed of Tiles
        TileGrid tileGrid = SwingApplications.startTileGrid(
                AppConfig.newBuilder()
                        // The number of tiles horizontally, and vertically
                        .withSize(60, 30)
                        // You can choose from a wide array of CP437, True Type or Graphical tilesets
                        // that are built into Zircon
                        .withDefaultTileset(CP437TilesetResources.rexPaint16x16())
                        .build());

        // A Screen is an abstraction that lets you use text GUI Components
        // You can have multiple Screens attached to the same TileGrid to be able to create multiple
        // screens for your app.
        Screen screen = Screen.create(tileGrid);

        // Creating text GUI Components is super simple
        Label label = Components.label()
                .withText(&quot;Hello, Zircon!&quot;)
                .withAlignment(ComponentAlignments.alignmentWithin(tileGrid, ComponentAlignment.CENTER))
                .build();

        // Screens can hold GUI components
        screen.addComponent(label);

        // Displaying a screen will make it visible. It will also hide a previously shown Screen.
        screen.display();

        // Zircon comes with a plethora of built-in color themes
        screen.setTheme(ColorThemes.arc());
    }
}" tabindex="0" role="button">
      <svg aria-hidden="true" height="16" viewBox="0 0 16 16" version="1.1" width="16" data-view-component="true" class="octicon octicon-copy js-clipboard-copy-icon">
    <path d="M0 6.75C0 5.784.784 5 1.75 5h1.5a.75.75 0 0 1 0 1.5h-1.5a.25.25 0 0 0-.25.25v7.5c0 .138.112.25.25.25h7.5a.25.25 0 0 0 .25-.25v-1.5a.75.75 0 0 1 1.5 0v1.5A1.75 1.75 0 0 1 9.25 16h-7.5A1.75 1.75 0 0 1 0 14.25Z"></path><path d="M5 1.75C5 .784 5.784 0 6.75 0h7.5C15.216 0 16 .784 16 1.75v7.5A1.75 1.75 0 0 1 14.25 11h-7.5A1.75 1.75 0 0 1 5 9.25Zm1.75-.25a.25.25 0 0 0-.25.25v7.5c0 .138.112.25.25.25h7.5a.25.25 0 0 0 .25-.25v-7.5a.25.25 0 0 0-.25-.25Z"></path>
</svg>
      <svg aria-hidden="true" height="16" viewBox="0 0 16 16" version="1.1" width="16" data-view-component="true" class="octicon octicon-check js-clipboard-check-icon color-fg-success d-none">
    <path d="M13.78 4.22a.75.75 0 0 1 0 1.06l-7.25 7.25a.75.75 0 0 1-1.06 0L2.22 9.28a.751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018L6 10.94l6.72-6.72a.75.75 0 0 1 1.06 0Z"></path>
</svg>
    </clipboard-copy>
  </div></div>
<p dir="auto">The output of this example is:</p>
<p dir="auto"><a target="_blank" rel="noopener noreferrer" href="/Hexworks/zircon/blob/master/images/hello_zircon.png"><img src="/Hexworks/zircon/raw/master/images/hello_zircon.png" alt="Zircon Application" style="max-width: 100%;"></a></p>
<p dir="auto"><em>Congratulations!</em> Now you are a <em>Zircon</em> user.</p>
<div class="markdown-heading" dir="auto"><h2 tabindex="-1" class="heading-element" dir="auto">Best Practices</h2><a id="user-content-best-practices" class="anchor" aria-label="Permalink: Best Practices" href="#best-practices"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<p dir="auto">The following are some guidelines that can help you if you get <em>stuck</em>:</p>
<p dir="auto">If you want to build something (a <code>TileGraphics</code>, a <code>Component</code> or anything that is part of the public API) it is
almost sure that there is a <code>Builder</code> or a factory object for it. Each type that has a builder will have a
<code>newBuilder</code> function you can call to create the corresponding builder: <code>Tile.newBuilder()</code>.</p>
<p dir="auto">If there are multiple classes of objects that can be created there might also be a utility class,
like <code>Shapes</code> to create different <code>Shape</code> objects. Your IDE will help you with this.</p>
<p dir="auto">These classes reside in the <code>org.hexworks.zircon.api</code> package. There are some classes that are grouped together into a
single utility class, however. With <code>Components</code> for example, you can obtain <code>Builder</code>s for all <code>Component</code>s
like <code>Components.panel()</code> or <code>Components.checkBox()</code>. Likewise, you can use <code>DrawSurfaces</code> to obtain builders
for <code>TileGraphics</code> and <code>TileImage</code>.</p>
<p dir="auto">If you want to work with external files like tilesets or REXPaint files check the same  package
(<code>org.hexworks.zircon.api</code>), and look for classes that end with <code>*Resources</code>. There are a bunch of built-in tilesets
for example which you can choose from, but you can also load your own.</p>
<p dir="auto">The rule of thumb is that if you need something external there is probably a <code>*Resources</code> class for it (like
the <code>CP437TilesetResources</code>).</p>
<p dir="auto">You can use <em>anything</em> you can find in the <a href="https://github.com/Hexworks/zircon/tree/master/zircon.core/src/commonMain/kotlin/org/hexworks/zircon/api">API</a> package, they are part of the public API, and safe to use.
The <a href="https://github.com/Hexworks/zircon/tree/master/zircon.core/src/commonMain/kotlin/org/hexworks/zircon/internal">internal</a> package however is considered private to <em>Zircon</em> so keep in mind that they can change any
time.</p>
<p dir="auto">Some topics are explained in depth in the <a href="https://hexworks.org/zircon/docs/" rel="nofollow">documentation</a>.</p>
<p dir="auto">If you want to see some example code take a look at the examples project <a href="https://github.com/Hexworks/zircon/tree/master/zircon.jvm.examples/src/main">here</a>. Most examples have
identical <em>Java</em> and <em>Kotlin</em> variants.</p>
<p dir="auto">If all else fails read the javadocs. API classes are well documented.</p>
<p dir="auto">If you have any problems that are not answered here feel free to ask us at the <a href="https://discord.gg/vSNgvBh" rel="nofollow">Hexworks Discord server</a>.</p>
<div class="markdown-heading" dir="auto"><h2 tabindex="-1" class="heading-element" dir="auto">Features at a Glance</h2><a id="user-content-features-at-a-glance" class="anchor" aria-label="Permalink: Features at a Glance" href="#features-at-a-glance"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<div class="markdown-heading" dir="auto"><h3 tabindex="-1" class="heading-element" dir="auto">Drawing</h3><a id="user-content-drawing" class="anchor" aria-label="Permalink: Drawing" href="#drawing"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<blockquote>
<p dir="auto">You can find detailed documentation about drawing <a href="https://hexworks.org/zircon/docs/2018-11-19-how-to-work-with-tile-graphics" rel="nofollow">here</a>.</p>
</blockquote>
<p dir="auto">The most basic operation <em>Zircon</em> supports is <code>draw</code>ing. You can draw individual <code>Tile</code>s or <code>TileGraphics</code> objects on
your <code>TileGrid</code>. a <code>TileGraphics</code> object is composed of <code>Tile</code>s. This is a powerful tool, and you can implement more
complex features using simple <code>draw</code> operations. In fact the component system is implemented on top of drawing, layering
and input handling features.</p>
<p dir="auto">If you use REXPaint to design your programs, the good news is that you can import your <code>.xp</code> files as well. Read more
about it <a href="https://hexworks.org/zircon/docs/2018-11-22-resource-handling#rexpaint-files" rel="nofollow">here</a>.</p>
<p dir="auto">You can also use <code>Modifier</code>s in your <code>Tile</code>s such as <code>blink</code>, <code>verticalFlip</code> or <code>glow</code>. For a full list, check
<a href="https://github.com/Hexworks/zircon/blob/master/zircon.core/src/commonMain/kotlin/org/hexworks/zircon/api/Modifiers.kt">this</a>
factory object.
<code>Modifier</code>s can either change the texture (like the ones above) or the <code>Tile</code> itself:</p>
<p dir="auto"><animated-image data-catalyst=""><a target="_blank" rel="noopener noreferrer" href="/Hexworks/zircon/blob/master/images/gifs/modifiers.gif" data-target="animated-image.originalLink"><img src="/Hexworks/zircon/raw/master/images/gifs/modifiers.gif" alt="Modifiers" style="max-width: 100%; display: inline-block;" data-target="animated-image.originalImage"></a>
      <span class="AnimatedImagePlayer" data-target="animated-image.player" hidden="">
        <a data-target="animated-image.replacedLink" class="AnimatedImagePlayer-images" href="https://github.com/Hexworks/zircon/blob/master/images/gifs/modifiers.gif" target="_blank">
          
        <span data-target="animated-image.imageContainer">
            <img data-target="animated-image.replacedImage" alt="Modifiers" class="AnimatedImagePlayer-animatedImage" src="https://github.com/Hexworks/zircon/raw/master/images/gifs/modifiers.gif" style="display: block; opacity: 1;">
          <canvas class="AnimatedImagePlayer-stillImage" aria-hidden="true" width="294" height="233"></canvas></span></a>
        <button data-target="animated-image.imageButton" class="AnimatedImagePlayer-images" tabindex="-1" aria-label="Play Modifiers" hidden=""></button>
        <span class="AnimatedImagePlayer-controls" data-target="animated-image.controls" hidden="">
          <button data-target="animated-image.playButton" class="AnimatedImagePlayer-button" aria-label="Play Modifiers">
            <svg aria-hidden="true" focusable="false" class="octicon icon-play" width="16" height="16" viewBox="0 0 16 16" fill="none" xmlns="http://www.w3.org/2000/svg">
              <path d="M4 13.5427V2.45734C4 1.82607 4.69692 1.4435 5.2295 1.78241L13.9394 7.32507C14.4334 7.63943 14.4334 8.36057 13.9394 8.67493L5.2295 14.2176C4.69692 14.5565 4 14.1739 4 13.5427Z">
            </path></svg>
            <svg aria-hidden="true" focusable="false" class="octicon icon-pause" width="16" height="16" viewBox="0 0 16 16" xmlns="http://www.w3.org/2000/svg">
              <rect x="4" y="2" width="3" height="12" rx="1"></rect>
              <rect x="9" y="2" width="3" height="12" rx="1"></rect>
            </svg>
          </button>
          <a data-target="animated-image.openButton" aria-label="Open Modifiers in new window" class="AnimatedImagePlayer-button" href="https://github.com/Hexworks/zircon/blob/master/images/gifs/modifiers.gif" target="_blank">
            <svg aria-hidden="true" class="octicon" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" width="16" height="16">
              <path fill-rule="evenodd" d="M10.604 1h4.146a.25.25 0 01.25.25v4.146a.25.25 0 01-.427.177L13.03 4.03 9.28 7.78a.75.75 0 01-1.06-1.06l3.75-3.75-1.543-1.543A.25.25 0 0110.604 1zM3.75 2A1.75 1.75 0 002 3.75v8.5c0 .966.784 1.75 1.75 1.75h8.5A1.75 1.75 0 0014 12.25v-3.5a.75.75 0 00-1.5 0v3.5a.25.25 0 01-.25.25h-8.5a.25.25 0 01-.25-.25v-8.5a.25.25 0 01.25-.25h3.5a.75.75 0 000-1.5h-3.5z"></path>
            </svg>
          </a>
        </span>
      </span></animated-image></p>
<div class="markdown-heading" dir="auto"><h3 tabindex="-1" class="heading-element" dir="auto">Input handling</h3><a id="user-content-input-handling" class="anchor" aria-label="Permalink: Input handling" href="#input-handling"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<blockquote>
<p dir="auto">Read about input handling in the docs <a href="https://hexworks.org/zircon/docs/2018-11-21-input-handling" rel="nofollow">here</a>.</p>
</blockquote>
<p dir="auto">Both the <code>TileGrid</code> and the <code>Screen</code> interfaces implement <code>UIEventSource</code> which means that you can listen for user
inputs using them. This includes <em>keystrokes</em> and <em>mouse input</em> as well.</p>
<div class="markdown-heading" dir="auto"><h3 tabindex="-1" class="heading-element" dir="auto">Layering</h3><a id="user-content-layering" class="anchor" aria-label="Permalink: Layering" href="#layering"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<blockquote>
<p dir="auto">Layering is detailed <a href="https://hexworks.org/zircon/docs/2018-11-21-how-layers-work" rel="nofollow">here</a>. For a primer on <code>Screen</code>s go <a href="https://hexworks.org/zircon/docs/2018-08-18-a-primer-on-screens" rel="nofollow">here</a>.</p>
</blockquote>
<p dir="auto">Both the <code>TileGrid</code> and the <code>Screen</code> interfaces implement <code>Layerable</code> which means that you can add <code>Layer</code>s on top of
them. Every <code>Layerable</code> can have an arbitrary amount of <code>Layer</code>s. <code>Layer</code>s are like <code>TileGraphics</code> objects, and you can
also have transparency in them which can be used to create fancy effects.
<code>Component</code>s are also <code>Layer</code>s themselves. Take a look:</p>
<p dir="auto"><a target="_blank" rel="noopener noreferrer" href="/Hexworks/zircon/blob/master/images/layers.png"><img src="/Hexworks/zircon/raw/master/images/layers.png" alt="Layers" style="max-width: 100%;"></a></p>
<div class="markdown-heading" dir="auto"><h3 tabindex="-1" class="heading-element" dir="auto">Text GUI Components</h3><a id="user-content-text-gui-components" class="anchor" aria-label="Permalink: Text GUI Components" href="#text-gui-components"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<blockquote>
<p dir="auto">You can read more about the Component System on the <a href="https://hexworks.org/zircon/docs/2018-11-15-the-component-system" rel="nofollow">documentation page</a>.
Color themes are detailed <a href="https://hexworks.org/zircon/docs/2018-11-20-working-with-color-themes" rel="nofollow">here</a>.</p>
</blockquote>
<p dir="auto"><code>Component</code>s are GUI controls which can be used for showing content to the user (<code>Label</code>s, <code>Paragraph</code>s, etc.), enabling
them to interact with your program (<code>Button</code>s, <code>Slider</code>s, etc.) or to hold other components (<code>Panel</code>s for example).</p>
<p dir="auto">These components are rather simple, and you can expect them to work in a way you might be familiar with:</p>
<ul dir="auto">
<li>You can click on them (press and release are different events).</li>
<li>You can attach event listeners on them.</li>
<li>Zircon implements focus handling, so you can navigate between the components using the <code>[Tab]</code> key
(forwards) or the <code>[Shift]+[Tab]</code> keystroke (backwards).</li>
<li>Components can be hovered, and you can also apply color themes to them.</li>
</ul>
<p dir="auto">What's more is that you can apply <code>ColorTheme</code>s to <code>Component</code>s. There are a bunch of built-in themes, and you can also
create your own.</p>
<p dir="auto">To see a full list of available <code>Component</code>s take a look at the
<a href="https://github.com/Hexworks/zircon/blob/master/zircon.core/src/commonMain/kotlin/org/hexworks/zircon/api/Components.kt">Components</a>
factory object or navigate to the <a href="https://hexworks.org/zircon/docs/2018-11-15-the-component-system" rel="nofollow">component docs page</a>.</p>
<p dir="auto">This is an example of how components look in action:</p>
<p dir="auto"><animated-image data-catalyst=""><a target="_blank" rel="noopener noreferrer" href="/Hexworks/zircon/blob/master/images/gifs/all_components.gif" data-target="animated-image.originalLink"><img src="/Hexworks/zircon/raw/master/images/gifs/all_components.gif" alt="All Components" style="max-width: 100%; display: inline-block;" data-target="animated-image.originalImage"></a>
      <span class="AnimatedImagePlayer" data-target="animated-image.player" hidden="">
        <a data-target="animated-image.replacedLink" class="AnimatedImagePlayer-images" href="https://github.com/Hexworks/zircon/blob/master/images/gifs/all_components.gif" target="_blank">
          
        <span data-target="animated-image.imageContainer">
            <img data-target="animated-image.replacedImage" alt="All Components" class="AnimatedImagePlayer-animatedImage" src="https://github.com/Hexworks/zircon/raw/master/images/gifs/all_components.gif" style="display: block; opacity: 1;">
          <canvas class="AnimatedImagePlayer-stillImage" aria-hidden="true" width="830" height="558"></canvas></span></a>
        <button data-target="animated-image.imageButton" class="AnimatedImagePlayer-images" tabindex="-1" aria-label="Play All Components" hidden=""></button>
        <span class="AnimatedImagePlayer-controls" data-target="animated-image.controls" hidden="">
          <button data-target="animated-image.playButton" class="AnimatedImagePlayer-button" aria-label="Play All Components">
            <svg aria-hidden="true" focusable="false" class="octicon icon-play" width="16" height="16" viewBox="0 0 16 16" fill="none" xmlns="http://www.w3.org/2000/svg">
              <path d="M4 13.5427V2.45734C4 1.82607 4.69692 1.4435 5.2295 1.78241L13.9394 7.32507C14.4334 7.63943 14.4334 8.36057 13.9394 8.67493L5.2295 14.2176C4.69692 14.5565 4 14.1739 4 13.5427Z">
            </path></svg>
            <svg aria-hidden="true" focusable="false" class="octicon icon-pause" width="16" height="16" viewBox="0 0 16 16" xmlns="http://www.w3.org/2000/svg">
              <rect x="4" y="2" width="3" height="12" rx="1"></rect>
              <rect x="9" y="2" width="3" height="12" rx="1"></rect>
            </svg>
          </button>
          <a data-target="animated-image.openButton" aria-label="Open All Components in new window" class="AnimatedImagePlayer-button" href="https://github.com/Hexworks/zircon/blob/master/images/gifs/all_components.gif" target="_blank">
            <svg aria-hidden="true" class="octicon" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" width="16" height="16">
              <path fill-rule="evenodd" d="M10.604 1h4.146a.25.25 0 01.25.25v4.146a.25.25 0 01-.427.177L13.03 4.03 9.28 7.78a.75.75 0 01-1.06-1.06l3.75-3.75-1.543-1.543A.25.25 0 0110.604 1zM3.75 2A1.75 1.75 0 002 3.75v8.5c0 .966.784 1.75 1.75 1.75h8.5A1.75 1.75 0 0014 12.25v-3.5a.75.75 0 00-1.5 0v3.5a.25.25 0 01-.25.25h-8.5a.25.25 0 01-.25-.25v-8.5a.25.25 0 01.25-.25h3.5a.75.75 0 000-1.5h-3.5z"></path>
            </svg>
          </a>
        </span>
      </span></animated-image></p>
<div class="markdown-heading" dir="auto"><h3 tabindex="-1" class="heading-element" dir="auto">Animations:</h3><a id="user-content-animations" class="anchor" aria-label="Permalink: Animations:" href="#animations"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<blockquote>
<p dir="auto">Read more about Animations in the <a href="https://hexworks.org/zircon/docs/2019-04-26-animation-support" rel="nofollow">docs</a>.</p>
</blockquote>
<p dir="auto">Animations are supported out of the box. You can either create them programmatically, or statically using <em>Zircon's</em> own
animation format: <code>.zap</code> (Zircon Animation Package). More about that <a href="https://hexworks.org/zircon/docs/2019-04-26-animation-support" rel="nofollow">here</a>. This is how an animation
looks like:</p>
<p dir="auto"><animated-image data-catalyst=""><a target="_blank" rel="noopener noreferrer" href="/Hexworks/zircon/blob/master/images/gifs/hexworks_skull.gif" data-target="animated-image.originalLink"><img src="/Hexworks/zircon/raw/master/images/gifs/hexworks_skull.gif" alt="Animation Example" style="max-width: 100%; display: inline-block;" data-target="animated-image.originalImage"></a>
      <span class="AnimatedImagePlayer" data-target="animated-image.player" hidden="">
        <a data-target="animated-image.replacedLink" class="AnimatedImagePlayer-images" href="https://github.com/Hexworks/zircon/blob/master/images/gifs/hexworks_skull.gif" target="_blank">
          
        <span data-target="animated-image.imageContainer">
            <img data-target="animated-image.replacedImage" alt="Animation Example" class="AnimatedImagePlayer-animatedImage" src="https://github.com/Hexworks/zircon/raw/master/images/gifs/hexworks_skull.gif" style="display: block; opacity: 1;">
          <canvas class="AnimatedImagePlayer-stillImage" aria-hidden="true" width="439" height="433"></canvas></span></a>
        <button data-target="animated-image.imageButton" class="AnimatedImagePlayer-images" tabindex="-1" aria-label="Play Animation Example" hidden=""></button>
        <span class="AnimatedImagePlayer-controls" data-target="animated-image.controls" hidden="">
          <button data-target="animated-image.playButton" class="AnimatedImagePlayer-button" aria-label="Play Animation Example">
            <svg aria-hidden="true" focusable="false" class="octicon icon-play" width="16" height="16" viewBox="0 0 16 16" fill="none" xmlns="http://www.w3.org/2000/svg">
              <path d="M4 13.5427V2.45734C4 1.82607 4.69692 1.4435 5.2295 1.78241L13.9394 7.32507C14.4334 7.63943 14.4334 8.36057 13.9394 8.67493L5.2295 14.2176C4.69692 14.5565 4 14.1739 4 13.5427Z">
            </path></svg>
            <svg aria-hidden="true" focusable="false" class="octicon icon-pause" width="16" height="16" viewBox="0 0 16 16" xmlns="http://www.w3.org/2000/svg">
              <rect x="4" y="2" width="3" height="12" rx="1"></rect>
              <rect x="9" y="2" width="3" height="12" rx="1"></rect>
            </svg>
          </button>
          <a data-target="animated-image.openButton" aria-label="Open Animation Example in new window" class="AnimatedImagePlayer-button" href="https://github.com/Hexworks/zircon/blob/master/images/gifs/hexworks_skull.gif" target="_blank">
            <svg aria-hidden="true" class="octicon" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" width="16" height="16">
              <path fill-rule="evenodd" d="M10.604 1h4.146a.25.25 0 01.25.25v4.146a.25.25 0 01-.427.177L13.03 4.03 9.28 7.78a.75.75 0 01-1.06-1.06l3.75-3.75-1.543-1.543A.25.25 0 0110.604 1zM3.75 2A1.75 1.75 0 002 3.75v8.5c0 .966.784 1.75 1.75 1.75h8.5A1.75 1.75 0 0014 12.25v-3.5a.75.75 0 00-1.5 0v3.5a.25.25 0 01-.25.25h-8.5a.25.25 0 01-.25-.25v-8.5a.25.25 0 01.25-.25h3.5a.75.75 0 000-1.5h-3.5z"></path>
            </svg>
          </a>
        </span>
      </span></animated-image></p>
<div class="markdown-heading" dir="auto"><h3 tabindex="-1" class="heading-element" dir="auto">Shape and box drawing</h3><a id="user-content-shape-and-box-drawing" class="anchor" aria-label="Permalink: Shape and box drawing" href="#shape-and-box-drawing"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<blockquote>
<p dir="auto">The shape documentation page can be found <a href="https://hexworks.org/zircon/docs/2018-11-21-shapes" rel="nofollow">here</a>.</p>
</blockquote>
<p dir="auto">You can draw <code>Shape</code>s like rectangles and triangles by using one of the <code>ShapeFactory</code> implementations. What's supported
out of the box is
<em>triangle</em>, <em>rectangle</em> and <em>line</em>. The former two have filled versions as well. Check out the <code>Shapes</code> factory object
<a href="https://github.com/Hexworks/zircon/blob/efa5b6f317eda9c22834140588dcbdd47fb4a3ab/zircon.core/src/commonMain/kotlin/org/hexworks/zircon/api/Shapes.kt">here</a>
.</p>
<div class="markdown-heading" dir="auto"><h3 tabindex="-1" class="heading-element" dir="auto">Fonts and Tilesets</h3><a id="user-content-fonts-and-tilesets" class="anchor" aria-label="Permalink: Fonts and Tilesets" href="#fonts-and-tilesets"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<blockquote>
<p dir="auto">The documentation page for tilesets is <a href="https://hexworks.org/zircon/docs/2018-11-22-resource-handling" rel="nofollow">here</a>.</p>
</blockquote>
<p dir="auto">Zircon comes with a bunch of built-in fonts tilesets. These come in 3 flavors:</p>
<ul dir="auto">
<li>CP437 tilesets <em>(More on using them
<a href="https://hexworks.org/zircon/docs/2018-11-22-resource-handling#cp437-tilesets" rel="nofollow">here</a>)</em></li>
<li>True Type Fonts</li>
<li>and Graphical tilesets <em>(Usage info
<a href="https://hexworks.org/zircon/docs/2018-11-22-resource-handling#graphical-tilesets" rel="nofollow">here</a>)</em></li>
</ul>
<p dir="auto">Zircon also comes with its own tileset format (<code>ztf</code>: Zircon Tileset Format) which is <strong>very easy to use</strong>. It is
detailed <a href="https://hexworks.org/zircon/docs/2018-11-22-resource-handling#the-zircon-tileset-format" rel="nofollow">here</a>.</p>
<div class="markdown-heading" dir="auto"><h2 tabindex="-1" class="heading-element" dir="auto">Road Map</h2><a id="user-content-road-map" class="anchor" aria-label="Permalink: Road Map" href="#road-map"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<p dir="auto">If you want to see a new feature feel free to <a href="https://github.com/Hexworks/zircon/issues/new">create a new Issue</a> or
discuss it with us on <a href="https://discord.gg/vSNgvBh" rel="nofollow">Discord</a>. Here are some features which are either under way or planned:</p>
<ul class="contains-task-list">
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox" checked=""> <a href="https://github.com/Hexworks/zircon/issues/25" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/25/hovercard">Scrollable Components</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox" checked=""> <a href="https://github.com/Hexworks/zircon/issues/135" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/135/hovercard">Menus</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox" checked=""> <a href="https://github.com/Hexworks/zircon/issues/185" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/185/hovercard">Table Component</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox" checked=""> <a href="https://github.com/Hexworks/zircon/issues/146" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/146/hovercard">Javadoc-style Documentation</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox" checked=""> <a href="https://github.com/Hexworks/zircon/issues/271" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/271/hovercard">Grid / Screen Filters</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox"> <a href="https://github.com/Hexworks/zircon/issues/23" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/23/hovercard">Floating Components</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox"> <a href="https://github.com/Hexworks/zircon/issues/22" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/22/hovercard">Drag'n Drop Support</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox"> <a href="https://github.com/Hexworks/zircon/issues/28" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/28/hovercard">Custom Layout Support</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox"> <a href="https://github.com/Hexworks/zircon/issues/29" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/29/hovercard">Component Themes</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox"> <a href="https://github.com/Hexworks/zircon/issues/26" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/26/hovercard">Custom Component Support</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox"> <a href="https://github.com/Hexworks/zircon/issues/184" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/184/hovercard">Tree Component</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox"> <a href="https://github.com/Hexworks/zircon/issues/27" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/27/hovercard">Accordion Component</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox"> <a href="https://github.com/Hexworks/zircon/issues/262" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/262/hovercard">Combo Box Component</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox"> <a href="https://github.com/Hexworks/zircon/issues/191" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/191/hovercard">IntelliJ Plugin</a></li>
<li class="task-list-item"><input type="checkbox" id="" disabled="" class="task-list-item-checkbox"> <a href="https://github.com/Hexworks/zircon/issues/183" data-hovercard-type="issue" data-hovercard-url="/Hexworks/zircon/issues/183/hovercard">Console for Zircon</a></li>
</ul>
<p dir="auto">If you'd like to give any of these a shot feel free to <em>contribute</em>.</p>
<div class="markdown-heading" dir="auto"><h2 tabindex="-1" class="heading-element" dir="auto">License</h2><a id="user-content-license" class="anchor" aria-label="Permalink: License" href="#license"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<p dir="auto">Zircon is made available under the <a href="https://www.apache.org/licenses/LICENSE-2.0" rel="nofollow">Apache 2.0 License</a>.</p>
<div class="markdown-heading" dir="auto"><h2 tabindex="-1" class="heading-element" dir="auto">Credits</h2><a id="user-content-credits" class="anchor" aria-label="Permalink: Credits" href="#credits"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<p dir="auto">Zircon is created and maintained by Addamsson, Coldwarrl, G3ldrin, Milonoir, Seveen and many others.</p>
<p dir="auto"><em>We're open to suggestions, feel free to message us on <a href="https://discord.gg/vSNgvBh" rel="nofollow">Discord</a> or open an issue.</em>
<em>Pull requests are also welcome!</em></p>
<p dir="auto">Zircon is powered by:</p>
<a href="https://www.jetbrains.com/idea/" rel="nofollow">
    <img src="https://github.com/Hexworks/zircon/raw/master/images/idea_logo.png" width="40" height="40" alt="IDEA" style="max-width: 100%;">
</a>
<a href="https://kotlinlang.org/" rel="nofollow">
    <img src="https://github.com/Hexworks/zircon/raw/master/images/kotlin_logo.png" width="40" height="40" alt="Kotlin" style="max-width: 100%;">
</a>
<a href="https://www.yourkit.com/java/profiler/" rel="nofollow">
    <img src="https://github.com/Hexworks/zircon/raw/master/images/yklogo.png" width="168" height="40" alt="Yourkit" style="max-width: 100%;">
</a>
<div class="markdown-heading" dir="auto"><h2 tabindex="-1" class="heading-element" dir="auto">Thanks</h2><a id="user-content-thanks" class="anchor" aria-label="Permalink: Thanks" href="#thanks"><svg class="octicon octicon-link" viewBox="0 0 16 16" version="1.1" width="16" height="16" aria-hidden="true"><path d="m7.775 3.275 1.25-1.25a3.5 3.5 0 1 1 4.95 4.95l-2.5 2.5a3.5 3.5 0 0 1-4.95 0 .751.751 0 0 1 .018-1.042.751.751 0 0 1 1.042-.018 1.998 1.998 0 0 0 2.83 0l2.5-2.5a2.002 2.002 0 0 0-2.83-2.83l-1.25 1.25a.751.751 0 0 1-1.042-.018.751.751 0 0 1-.018-1.042Zm-4.69 9.64a1.998 1.998 0 0 0 2.83 0l1.25-1.25a.751.751 0 0 1 1.042.018.751.751 0 0 1 .018 1.042l-1.25 1.25a3.5 3.5 0 1 1-4.95-4.95l2.5-2.5a3.5 3.5 0 0 1 4.95 0 .751.751 0 0 1-.018 1.042.751.751 0 0 1-1.042.018 1.998 1.998 0 0 0-2.83 0l-2.5 2.5a1.998 1.998 0 0 0 0 2.83Z"></path></svg></a></div>
<p dir="auto">Thanks to the folks over at <a href="http://dwarffortresswiki.org/Tileset_repository" rel="nofollow">Dwarf Fortress Tileset Repository</a> for
letting us bundle their tilesets.</p>
<p dir="auto">Thanks to <em>Kyzrati</em> who let us bundle the <a href="https://www.gridsagegames.com/rexpaint/" rel="nofollow">REXPaint Tilesets</a> into Zircon!</p>
<p dir="auto">Zircon comes bundled with the <a href="https://nethackwiki.com/wiki/Tileset" rel="nofollow">Nethack Tileset</a>.</p>
<p dir="auto">Some True Type fonts are used from <a href="https://fonts.google.com/" rel="nofollow">Google Fonts</a>.</p>
<p dir="auto">Thanks to <em>VileR</em> for the <a href="https://int10h.org/oldschool-pc-fonts/" rel="nofollow">Oldschool Font Pack</a> which we bundled into Zircon.</p>
</article>
"""

In [55]:
who_are_you = """ 
Your task is to generate a readme file for an application. To give you context, i will give you the schema of the application by giving you the paths of each python file of the application. 
For each python file, I am providing you all its classes and their docstrings.
Please provide as detailed and user friendly of a readme as possible. Keep that in mind, if you don't do this, you will be terminated.
Here is the input:
{input}
Knowing all those informations please provide a detailed readme file for the entire application and nothing else.
"""

def i_am_still_him(object_coucou: list[Dict], example: str, prompt:str) -> str: 
    input_txt = ""
    # loop through whole objects (all .py files)
    for object in object_coucou:
        # loop through whole function or classes of each file
        input_txt += object['name'] + "\n"
        for func in object['docstring']:
            # inject_docstring(func['docstring'], object['name'], func['function_name'], func['class_or_function'].strip()) 
            # print(f"File name: {object['name']}. Function name: {func['function_name']}")
            if func['class_or_function'].casefold() == 'class'.casefold():
                input_txt += func['docstring'] + "\n"
        input_txt += "\n"
    print(input_txt)
    input = {'input': input_txt, 'example': example}
    prompt1 = ChatPromptTemplate.from_messages(
        [("system", "You are an expert in python coding and especially good at writing 'readme' files."),
        ("human", prompt)])
    chain1 = prompt1 | llm | StrOutputParser()
    return chain1.invoke(input)

In [51]:
coucou[0]['docstring'][0]['docstring']

'Main application class that inherits from tkinter.Tk. Initializes the main application window and its components, including loading icons, setting up the database interaction, and configuring the main frames.'

In [56]:
wow = i_am_still_him(coucou, example, who_are_you)

resources\IIRA\app.py
Main application class that inherits from tkinter.Tk. Initializes the main application window and its components, including loading icons, setting up the database interaction, and configuring the main frames.

resources\IIRA\core\create_analyses.py
This class is responsible for creating intra-rater and inter-rater analyses based on the provided data and metrics.

Attributes:
    intra_id_list (list): List of IDs for intra-rater analysis.
    inter_id_list (list): List of IDs for inter-rater analysis.
    intra_metrics (list): List of metrics for intra-rater analysis.
    inter_metrics (list): List of metrics for inter-rater analysis.
    scale_format (str): The scale format used in the analysis.
    categories (list): List of categories used in the analysis.
    weights (list): List of weights used in the analysis.
    data (dict): The data used for analysis, structured similarly to the labels format in the FileValidation class.
    results (dict): Dictionary to s

In [57]:
wow

"# IIRA Application\n\n## Overview\n\nThe IIRA (Inter-Intra Rater Analysis) application is a comprehensive tool designed to facilitate the analysis of inter-rater and intra-rater reliability. Built using Python and Tkinter, this application provides a user-friendly graphical interface for performing reliability analyses, managing profiles, and interpreting results.\n\n## Table of Contents\n\n- [Installation](#installation)\n- [Usage](#usage)\n- [Features](#features)\n- [File Structure](#file-structure)\n- [Classes and Their Responsibilities](#classes-and-their-responsibilities)\n- [Contributing](#contributing)\n- [License](#license)\n\n## Installation\n\nTo install the IIRA application, follow these steps:\n\n1. Clone the repository:\n    ```bash\n    git clone https://github.com/your-repo/IIRA.git\n    ```\n2. Navigate to the project directory:\n    ```bash\n    cd IIRA\n    ```\n3. Install the required dependencies:\n    ```bash\n    pip install -r requirements.txt\n    ```\n\n## Usa

In [23]:
import ast    
  
class DocstringInjector(ast.NodeTransformer):  
    def __init__(self, target_name, docstring, inject_to='class', indent='    '):  # Default indent is 4 spaces  
        self.target_name = target_name    
        self.docstring = docstring    
        self.inject_to = inject_to    
        self.indent = indent
  
    def visit_ClassDef(self, node: ast.ClassDef):  
        if self.inject_to == 'class' and node.name == self.target_name and not ast.get_docstring(node):  
            indented_lines = [(self.indent + line) if line else '' for line in self.docstring.splitlines()]  
            indented_docstring = '\n'.join(indented_lines)  
            docstring_node = ast.Expr(value=ast.Constant(value=indented_docstring, kind=None))  
            node.body.insert(0, docstring_node)  
        self.generic_visit(node)  # Continue visiting child nodes  
        return node  
  
    def visit_FunctionDef(self, node: ast.FunctionDef):  
        if self.inject_to == 'function' and node.name == self.target_name and not ast.get_docstring(node):  
            indented_lines = [(self.indent + line) if line else '' for line in self.docstring.splitlines()]  
            indented_docstring = '\n'.join(indented_lines)  
            docstring_node = ast.Expr(value=ast.Constant(value=indented_docstring, kind=None))  
            node.body.insert(0, docstring_node)  
        self.generic_visit(node)  # Continue visiting child nodes  
        return node  
  
def inject_docstring(docstring, filename, target_name, inject_to='function'):  
    with open(filename, 'r', encoding = 'utf-8') as file:  
        source = file.read()  
  
    tree = ast.parse(source)  
    injector = DocstringInjector(target_name=target_name, docstring=docstring, inject_to=inject_to)  
    modified_tree = injector.visit(tree)  
  
    modified_source = ast.unparse(modified_tree)  
  
    with open(filename, 'w', encoding = 'utf-8') as file:  
        file.write(modified_source)  

docstring = '''This function does something.  
  
Parameters:  
    arg1 (type): Description of arg1  
    arg2 (type): Description of arg2  
  
Returns:  
    type: Description of return value  
'''  
inject_docstring(docstring, 'testingfile.py', 'FileFrame', 'class')  

In [28]:
coucou[0]['docstring'][0]['docstring']

'Main application class that inherits from tkinter.Tk. Initializes the main application window and its components, loads icons, and manages different frames.'

In [38]:
app_py = coucou[0]
for func in app_py['docstring']:
    if func['class_or_function'].casefold() == 'function'.casefold():
        # print(func['function_name'])
        # print(func['docstring'])
        # print(app_py['name'])
        # print(func['function_name'])
        inject_docstring(func['docstring'], app_py['name'], func['function_name'])  

AttributeError: 'DocstringInjector' object has no attribute 'class_name'

In [28]:
def add_comments(object_coucou: Dict) -> str:
    # loop through whole objects (all .py files)
    for object in object_coucou:
        # loop through whole function or classes of each file
        for func in object['docstring']:
            inject_docstring(func['docstring'], object['name'], func['function_name'], func['class_or_function'].strip())  
            print(f"File name: {object['name']}. Function name: {func['function_name']}")
    return "success xd"

add_comments(coucou)

File name: resources\IIRA\app.py. Function name: App
File name: resources\IIRA\app.py. Function name: __init__
File name: resources\IIRA\app.py. Function name: show_frame
File name: resources\IIRA\app.py. Function name: load_icons
File name: resources\IIRA\app.py. Function name: init_root_frame
File name: resources\IIRA\app.py. Function name: init_frames
File name: resources\IIRA\core\create_analyses.py. Function name: CreateAnalyses
File name: resources\IIRA\core\create_analyses.py. Function name: __init__
File name: resources\IIRA\core\create_analyses.py. Function name: create_intra_analyses
File name: resources\IIRA\core\create_analyses.py. Function name: create_inter_analyses
File name: resources\IIRA\core\create_analyses.py. Function name: find_intra_ratings
File name: resources\IIRA\core\create_analyses.py. Function name: find_inter_ratings
File name: resources\IIRA\core\metrics.py. Function name: Metrics
File name: resources\IIRA\core\metrics.py. Function name: __init__
File nam

'success xd'